# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tqdm import tqdm
# from tsa_functions import *
from tsa_tools import *  # See last cell

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

# Loading the Data

In [2]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_price = pd.read_csv('../data/m5/sell_prices.csv')
df_sales = pd.read_csv('../data/m5/sales_train_validation.csv')
df_sales1 = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')

In [61]:
full_df = (df_sales1.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]))

In [63]:
lvl9 = full_df.sum(level=["store_id", "dept_id"], axis=1)
lvl9.head()

store_id        CA_1                                                    \
dept_id    HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1 FOODS_2   
date                                                                     
2011-01-29       528        28         361         181     297     674   
2011-01-30       489         9         350         170     284     655   
2011-01-31       409         6         279         114     214     396   
2011-02-01       383         9         278         123     175     476   
2011-02-02       263         5         195         135     182     354   

store_id                CA_2                        ...    WI_2          \
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ... FOODS_1 FOODS_2   
date                                                ...                   
2011-01-29    2268       522        16         529  ...     191     155   
2011-01-30    2198       381        16         461  ...     184     128   
2011-01-31    1398       352        16         306  ...     154     199   
2011-02-01    1607       344         6         270  ...     205     244   
2011-02-02    1496       283        13         217  ...      96     147   

store_id                WI_3                                            \
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1   
date                                                                     
2011-01-29    1269       256        22         584         148     152   
2011-01-30    1121       342        14         541         195     138   
2011-01-31    1233       228        20         420         106     127   
2011-02-01    1564       183        11         327          94      98   
2011-02-02     724        70         4         151          53      87   

store_id                    
dept_id    FOODS_2 FOODS_3  
date                        
2011-01-29     583    2293  
2011-01-30     585    2383  
2011-01-31     575    1841  
2011-02-01     533    1965  
2011-02-02     340    1427  

[5 rows x 70 columns]

# Pre-procesing

## Pre-processing Full Dataset

In [40]:
# full_df_prep = (full_dflvl9.apply(lambda x: np.where(x < 10,  np.nan, x))
#                 .interpolate(method='linear', axis=0)).fillna(method='bfill')

## Pre-processing Lvl9

In [64]:
# No of values with less than 10 sales per day
lvl9[lvl9 < 10].count(axis=0).sum()

2403

In [65]:
df_lvl9 = (lvl9.apply(lambda x: np.where(x < 10,  np.nan, x))
            .interpolate(method='linear', axis=0)).fillna(method='bfill')

In [66]:
# To check if there are still values less than 10
df_lvl9[df_lvl9 < 10].count(axis=0).sum()

0

# Q1. Baseline Methods

In [67]:
train = df_lvl9.iloc[ :1913,:]
test = df_lvl9.iloc[ 1913:,:]

In [68]:
base_forcast = {}
h = 28
m = 7


# for bm in baseline:
naive = []
snaive = []
ses = []
hl = []
ahl = []
for x in train.columns:
    naive.append(naivef(train[x], h))
    snaive.append(snaivef(train[x], h, m))

    model1 =  ETSModel(train[x].values).fit() # SES
    model2 =  ETSModel(train[x].values, trend="add", seasonal=None).fit() # Holt's Linear
    model3 =  ETSModel(train[x], error="add", trend="add", 
                        seasonal="add", damped_trend=False, 
                        seasonal_periods=m).fit() # Additive Holt-Winter

    ses.append(model1.forecast(h))
    hl.append(model2.forecast(h))
    ahl.append(model3.forecast(h))

base_forcast['Naive'] = naive
base_forcast['Seasonal Naive'] = snaive
base_forcast['SES'] = ses
base_forcast['Holt\'s Linear'] = hl
base_forcast['Additive Holt-Winters'] = ahl


In [69]:
rmsse_res = {}

for k in base_forcast.keys():
    rmsse_list = []
    for x in range(len(train.columns)):
        rmsse_list.append(rmsse(test[train.columns[x]], 
                                base_forcast[k][x], 
                                train[train.columns[x]]))
    rmsse_res[k] = rmsse_list


In [ ]:
pd.set_option('max_rows', None)
lvl9_rmsse = pd.DataFrame(rmsse_res, index=train.columns)

lvl9_weights = (df_weights[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

lvl9_fin = pd.concat([lvl9_rmsse, lvl9_weights], axis=1)
lvl9_fin


Naive  Seasonal Naive     SES  Holt's Linear  \
CA_1 FOODS_1      0.9319          0.7312  0.9179         0.9362   
     FOODS_2      2.0535          0.8269  2.0534         2.3136   
     FOODS_3      1.7113          0.4944  1.0825         1.0185   
     HOBBIES_1    1.4583          0.7619  0.8819         0.8810   
     HOBBIES_2    1.9340          1.1429  0.8831         0.8748   
     HOUSEHOLD_1  2.1052          0.5172  1.1514         1.1636   
     HOUSEHOLD_2  2.2997          0.5228  1.2208         1.1789   
CA_2 FOODS_1      1.2557          1.1311  1.3875         1.3786   
     FOODS_2      4.2936          1.4906  2.3041         2.2415   
     FOODS_3      2.5830          0.6046  1.4502         1.4114   
     HOBBIES_1    1.2053          0.7055  1.1179         1.1180   
     HOBBIES_2    1.3546          1.4893  1.3887         1.3790   
     HOUSEHOLD_1  2.0287          0.6373  1.3960         1.3853   
     HOUSEHOLD_2  1.9340          0.5920  1.4625         1.4534   
CA_3 FOODS_1      1.1674          1.5874  1.3966         1.3892   
     FOODS_2      1.8816          0.7316  1.7271         1.7429   
     FOODS_3      1.5710          0.4353  0.8908         1.5877   
     HOBBIES_1    0.9370          0.6647  0.6999         0.7018   
     HOBBIES_2    1.1984          1.6617  1.5530         1.5433   
     HOUSEHOLD_1  2.6217          0.7564  1.1273         1.1536   
     HOUSEHOLD_2  2.1039          0.8808  1.4258         1.4169   
CA_4 FOODS_1      1.0167          1.5664  1.3857         1.3656   
     FOODS_2      1.2989          0.7998  1.1609         1.1533   
     FOODS_3      1.4144          0.6385  0.9469         0.9431   
     HOBBIES_1    1.4556          1.1008  0.9956         0.9902   
     HOBBIES_2    2.1142          1.8563  1.8756         1.8751   
     HOUSEHOLD_1  3.4088          1.4737  1.2447         1.2169   
     HOUSEHOLD_2  1.3534          0.9567  1.0520         1.0386   
TX_1 FOODS_1      1.3344          1.7189  1.5566         1.5460   
     FOODS_2      1.1906          0.9520  0.8562         0.8561   
     FOODS_3      1.1678          0.8416  0.9161         0.8855   
     HOBBIES_1    1.6331          0.9169  0.9061         0.9046   
     HOBBIES_2    1.8823          1.6475  1.4296         1.4159   
     HOUSEHOLD_1  1.8399          0.8196  1.1676         1.1608   
     HOUSEHOLD_2  1.2045          1.2523  1.0488         1.0328   
TX_2 FOODS_1      1.4774          1.2816  1.2919         1.2839   
     FOODS_2      0.9424          0.6834  0.7532         0.7719   
     FOODS_3      0.8228          0.6226  0.7499         0.7481   
     HOBBIES_1    0.7979          0.8792  0.8789         0.8742   
     HOBBIES_2    0.7499          1.2548  0.9735         0.9685   
     HOUSEHOLD_1  0.8662          0.8529  0.8683         0.8712   
     HOUSEHOLD_2  1.3706          0.6559  0.9314         0.9346   
TX_3 FOODS_1      2.7332          2.2206  2.1542         2.1356   
     FOODS_2      1.3682          1.4195  1.4254         1.4277   
     FOODS_3      0.9672          1.1046  1.0692         1.0493   
     HOBBIES_1    1.5309          0.9025  0.9404         0.9370   
     HOBBIES_2    1.4339          1.7212  1.1186         1.1152   
     HOUSEHOLD_1  1.0738          1.0794  1.0248         1.0064   
     HOUSEHOLD_2  0.8605          1.3137  1.0128         0.9889   
WI_1 FOODS_1      0.9843          0.8454  1.0438         1.0400   
     FOODS_2      2.1064          1.2493  1.6349         1.6291   
     FOODS_3      1.6655          0.5440  1.1554         1.1465   
     HOBBIES_1    1.3685          0.5084  0.7533         0.7533   
     HOBBIES_2    1.1278          1.6404  1.2088         1.1990   
     HOUSEHOLD_1  1.7201          0.6488  1.0982         1.0990   
     HOUSEHOLD_2  1.7301          0.7509  1.1014         1.0997   
WI_2 FOODS_1      2.4445          2.3635  2.2284         2.2117   
     FOODS_2      2.4210          2.4925  2.4266         2.3733   
     FOODS_3      1.3513          1.5445  1.3989         1.3654   

In [73]:
lvl9_wrmsse = {'Naive': sum(lvl9_fin['Naive'] * lvl9_fin['Weight']),
               'S. Naive': sum(lvl9_fin['Seasonal Naive'] * 
                                lvl9_fin['Weight']),
               'SES': sum(lvl9_fin['SES'] * lvl9_fin['Weight']),
               'Holt\'s Linear': sum(lvl9_fin['Holt\'s Linear'] * 
                                      lvl9_fin['Weight']),
               'Additive Holt-Winters': sum(lvl9_fin['Additive Holt-Winters'] 
                                             * lvl9_fin['Weight'])}
pd.DataFrame.from_dict(lvl9_wrmsse, orient='index', columns=['WRMSSE'])

,WRMSSE
Naive,1.6286
S. Naive,0.9300
SES,1.2075
Holt's Linear,1.2367
Additive Holt-Winters,0.8549


In [72]:
k = naivef(train[('CA_1', 'HOBBIES_1')], 28)
rmsse(test[('CA_1', 'HOBBIES_1')], k, train[('CA_1', 'HOBBIES_1')])

1.458257712456372

# Q7: Top-Down Function

In [ ]:
def compute_topdown(df_pred, level, approach='AHP'):
    """Returns"""

In [ ]:
df_full